In [ ]:
%load_ext autoreload
%autoreload 2
%run -n main.py

# datasets

In [ ]:
# path = join_path(DATA_DIR, DATASET)
# !mkdir -p {path}

In [ ]:
# for name in DATASETS:
#     paths = (
#         join_path(CORUS_DATA_DIR, _)
#         for _ in CORUS_FILES[name]
#     )
#     records = (
#         record
#         for path in paths
#         for record in load_dataset(path)
#     )
#     records = log_progress(records, desc=name)
#     records = sample(records, 1000)

#     path = join_path(DATA_DIR, DATASET, name + JL + GZ)
#     items = as_jsons(records)
#     lines = format_jl(items)
#     dump_gz_lines(lines, path)

In [ ]:
datasets = {}
for name in DATASETS:
    path = join_path(DATA_DIR, DATASET, name + JL + GZ)
    lines = load_gz_lines(path)
    items = parse_jl(lines)
    datasets[name] = list(from_jsons(items, Markup))

# models

In [ ]:
# for name in MODELS:
#     path = join_path(DATA_DIR, name)
#     !mkdir -p {path}

## cpu

In [ ]:
# docker = docker_client()

In [ ]:
# for model_name in MODELS:
#     log(model_name)
#     model = MODELS[model_name]()
#     model.start(docker)
#     model.wait()

#     for dataset_name in DATASETS:
#         records = model.map(_.words for _ in datasets[dataset_name])
#         records = log_progress(records, desc=dataset_name)

#         path = join_path(DATA_DIR, model_name, dataset_name + JL + GZ)
#         items = as_jsons(records)
#         lines = format_jl(items)
#         dump_gz_lines(lines, path)
    
#     model.stop(docker)

## gpu

In [ ]:
# # for deeppavlov models rent gpu
# # select cuda 10.0, high bandwith
# !vast search offers | grep '1 x  GTX 1080 Ti'

In [ ]:
# # model = DeeppavlovModel()
# model = DeeppavlovBERTModel()

In [ ]:
# !vast create instance 380530 --image {model.image} --disk 30

In [ ]:
# !vast show instances

In [ ]:
# !ssh ssh4.vast.ai -p 16279 -l root -Nf -L {model.port}:localhost:{model.container_port}

In [ ]:
# for dataset_name in DATASETS:
#     records = datasets[dataset_name]
#     records = log_progress(records, desc=dataset_name)
#     records = model.map(_.words for _ in records)

#     path = join_path(DATA_DIR, model.name, dataset_name + JL + GZ)
#     items = as_jsons(records)
#     lines = format_jl(items)
#     dump_gz_lines(lines, path)

In [ ]:
# !vast destroy instance 496279

# score

In [ ]:
dataset_models = {}
for dataset in DATASETS:
    for model in MODELS:
        path = join_path(DATA_DIR, model, dataset + JL + GZ)
        lines = load_gz_lines(path)
        items = parse_jl(lines)
        dataset_models[dataset, model] = list(from_jsons(items, Markup))

In [ ]:
scores = {}
for dataset, model in log_progress(dataset_models):
    preds = dataset_models[dataset, model]
    targets = datasets[dataset]
    scores[dataset, model] = score_markups(preds, targets)

# report

In [ ]:
table = scores_report_table(scores, DATASETS, MODELS)
html = format_scores_report(table)
patch_readme(MORPH1, html, README)
HTML(html)

In [ ]:
BENCH = [
    Bench(
        UDPIPE,
        init=6.91,
        disk=45 * MB,
        ram=242 * MB,
        speed=56.2,
    ),
    Bench(
        SPACY,
        init=10.9,
        disk=89 * MB,
        ram=579 * MB,
        speed=30.6,
    ),
    Bench(
        MARU,
        init=15.8,
        disk=44 * MB,
        ram=370 * MB,
        speed=36.4,
    ),
    Bench(
        RNNMORPH,
        init=8.73,
        disk=10 * MB,
        ram=289 * MB,
        speed=16.6,
    ),
    Bench(
        DEEPPAVLOV,
        init=4,
        disk=32 * MB,
        ram=10 * GB,
        speed=90,
        device=GPU
    ),
    Bench(
        DEEPPAVLOV_BERT,
        init=20,
        disk=(706 + 687) * MB,  # BERT + model
        ram=8.5 * GB,
        speed=85,
        device=GPU
    ),
    Bench(
        RUPOSTAGGER,
        init=4.81,
        disk=2.7 * MB,
        ram=118 * MB,
        speed=48,
    ),
]

html = format_bench_report(BENCH, MODELS)
patch_readme(MORPH2, html, README)
HTML(html)